In [4]:
import numpy as np
import cv2
%config IPCompleter.greedy=True # test autocompletion with tab or tab+shift

def toggleInfo (frame, mode):
    font = cv2.FONT_HERSHEY_SIMPLEX
    color = [255,255,255]
    if mode == 3 or mode == 4:
        color = [255,0,255]
    
    # display info text according to selection
    cv2.putText(frame, "q = quit", (5, 18), font, 0.7, color, 2)
    cv2.putText(frame, "0 = reset filter", (5, 41), font, 0.7, color, 2)

    if (mode == 0):
        cv2.putText(frame, "1 = gauss", (5, 60), font, 0.7, color, 2)
        cv2.putText(frame, "2 = toggle color space", (5, 80), font, 0.7, color, 2)
        cv2.putText(frame, "3 = toggle threshold", (5, 100), font, 0.7, color, 2)
        cv2.putText(frame, "4 = edge extraction", (5, 120), font, 0.7, color, 2)
    elif (mode == 2):
        cv2.putText(frame, "2 = toggle next color space", (5, 60), font, 0.7, color, 2)
    elif (mode == 3):
        cv2.putText(frame, "3 = toggle threshold", (5, 60), font, 0.7, color, 2)
    return frame
   
img = cv2.imread('Lenna.png')
frame = img.copy()

#cap = cv2.VideoCapture(0)
mode = 0
toggle_colspace = 0
toggle_thresh = 0
ret1 = 0; ret2=0
# font
font = cv2.FONT_HERSHEY_SIMPLEX
color = [255,255,255]
colorOnBw = [255,0,255]

while(True):
    # Capture frame-by-frame
    #ret, frame = cap.read()

    # wait for key and switch to mode
    ch = cv2.waitKey(1) & 0xFF
    
    if ch == ord('0'):
        mode = 0 # reset filters
    elif ch == ord('1') and mode == 0:
        mode = 1 # gauss
    elif ch == ord('2') and (mode == 0 or mode == 2):
        mode = 2 # color space
        toggle_colspace +=1
        toggle_colspace %= 4
    elif ch == ord('3') and (mode == 0 or mode == 3):
        mode = 3 # thresholding
        toggle_thresh +=1
        toggle_thresh %= 2
   
    elif ch == ord('4') and mode == 0:
        mode = 4 # canny edge
    elif ch == ord('q'):
        break

    if mode == 0:
        frame = img.copy()
        frame = toggleInfo (frame, mode)
        
    elif mode == 1:
        frame = cv2.GaussianBlur(img, (9, 9), 0)
        frame = toggleInfo (frame, mode) 
        
    elif mode == 2:
        if toggle_colspace == 1: # HSV
            frame = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            cv2.putText(frame, "Color space: HSV", (5, 500), font, 0.7, color, 2)
        elif toggle_colspace == 2: # LAB
            frame = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            cv2.putText(frame, "Color space: LAB", (5, 500), font, 0.7, color, 2)
        elif toggle_colspace == 3: # YUV
            frame = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
            cv2.putText(frame, "Color space: YUV", (5, 500), font, 0.7, color, 2)
        elif toggle_colspace == 0: # YCrCb    
            frame = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
            cv2.putText(frame, "Color space: YCrCb", (5, 500), font, 0.7, color, 2)
            
        frame = toggleInfo (frame, mode) 
        
    elif mode == 3:
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # info: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html'
        if toggle_thresh == 1: # gaussian threshold
            frame = cv2.adaptiveThreshold(frame,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                          cv2.THRESH_BINARY,11,2)
            frame = cv2.cvtColor(frame,cv2.COLOR_GRAY2RGB)
            cv2.putText(frame, "Threshold: gaussian", (5, 500), font,0.7, colorOnBw, 2)
        elif toggle_thresh == 0: # Otsu threshold
            ret2,frame = cv2.threshold(frame,0,255,cv2.THRESH_OTSU)
            frame = cv2.cvtColor(frame,cv2.COLOR_GRAY2RGB)
            cv2.putText(frame, "Threshold: otsu", (5, 500), font,0.7, colorOnBw, 2)
     
        frame = toggleInfo (frame, mode)
        
    elif mode == 4:
        frame = cv2.Canny(img,100,200)
        frame = cv2.cvtColor(frame,cv2.COLOR_GRAY2RGB)
        cv2.putText(frame, "Edge: canny", (5, 500), font, 0.7, colorOnBw, 2)
        frame = toggleInfo (frame, mode)
    
    # Display the resulting frame
    cv2.imshow('Various filters', frame)

#print(ret1,ret2)

#When everything done, release the capture
#cap.release()
cv2.destroyAllWindows()